In [1]:
import gatenlp

def convert_to_gatenlp_doc(data):
    # Create a GATE document
    doc = gatenlp.Document()
    doc.text = data["input"].replace("[START_ENT]", "").replace("[END_ENT]", "")
    

    return doc

# Your dataset
dataset = {
    "id": 0,
    "input": "[START_ENT] Bandar Seri Begawan [END_ENT] 11 15 AFP ...",
    "output": [
        {
            "answer": "Bandar Seri Begawan",
            "provenance": [{"title": "Bandar Seri Begawan"}]
        }
    ],
    "meta": {
        "left_context": "",
        "right_context": "11 15 AFP The United States today Wednesday deemed ...",
        "mention": "Bandar Seri Begawan"
    },
    "candidates": ["Bandar Seri Begawan", "Brunei International Airport"]
}

#read jsonl file
import json

with open('./ace2004-test-kilt.jsonl') as f:
    data = [json.loads(line) for line in f]
  


# Convert the dataset to a GATE document
gate_doc = convert_to_gatenlp_doc(data[0])


# Print the GATE document
print(gate_doc)


FileNotFoundError: [Errno 2] No such file or directory: 'data.jsonl'